In [99]:
import datetime
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
import polars as pl
import sqlalchemy as sql
from datetime import date
import numpy as np

"""
This function generates a plotly graph using a standard stock graph (from yfinance) and then plots all of the insider transactions for the stock on the graphic. red circle mean sell and green means
buy. Upon hovering over the circles, you get a lot more information about the transaction. This notebook was my original creation and testing of the main graph shown in the dash application. 
"""
def main():
    # will have to implement checks to prevent sqli just because itll be fun to do so
    ticker = input('enter stock ticker: ').upper().strip()

    engine = sql.create_engine('sqlite:///real.db')
    con = engine.connect()
    query = f'SELECT * FROM insider_data WHERE ISSUERTRADINGSYMBOL = "{ticker}"'
    # initialize dataframe with results from the query
    db_df = pd.read_sql(query, con)

    # access oldest avail insider transaction date
    oldest_date = db_df['FILING_DATE'].min()
    # subtract 2 years from oldest inside trade 
    oldest_date = (datetime.datetime.strptime(oldest_date,"%Y-%m-%d")  - datetime.timedelta(days=365*1)).date()
    todays_date = date.today().strftime('%Y-%m-%d')
    # get stock data on the daily timeframe from the oldest avail insider (actions=True gives us info about stock split)
    stock_data = yf.download(ticker, start=oldest_date, end=todays_date, actions=True)
    
    # get all rows where stock splits are not = 0
    split = stock_data[stock_data['Stock Splits'] != 0]
    
    # if there are stock splits -> change the price per share to that stock close price on that given day (thus reflecting the stock split)
    if len(split) > 0:
        for index, row in db_df.iterrows():
            try:
                db_df.at[index, 'TRANS_PRICEPERSHARE'] = round(stock_data.loc[row['FILING_DATE'], 'Close'], 2)
            except KeyError:
                db_df.drop(index)
    
    #print(db_df.head())
    fig = go.Figure()

    # stock_data.index is the date, then the y axis is plotting the closing price
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], name='Stock Price'))

    # create a scatter plot for the insider trading info:
    insider_trace = go.Scatter(
        x=db_df['FILING_DATE'],
        y=db_df['TRANS_PRICEPERSHARE'],      #stock_data.loc[db_df['FILING_DATE'], 'Close'],
        mode='markers',
        marker=dict(
            color=db_df['TRANS_ACQUIRED_DISP_CD'].apply(lambda x: 'green' if x == 'A' else 'red'),
            size=10
        ),
        showlegend=False,
        hovertext=db_df.apply(lambda row: f"Insider Transaction Info:<br>Shares:{row['TRANS_SHARES']}<br>Avg Cost: ${row['TRANS_PRICEPERSHARE']}<br>Filed on {row['FILING_DATE']}", axis=1),
        hoverinfo='text'
    )
    # Add legend entries for buy and sell markers
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color='green', size=10), showlegend=True, name='Insider Buy'))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color='red', size=10), showlegend=True, name='Insider Sell'))

    # Add traces to the figure
    fig.add_trace(insider_trace)

    fig.update_layout(
    title="Insider Trades vs Stock Price - Accounting for Stock Splits",
    xaxis_title="Date",
    yaxis_title="Price ($)",
    title_x=0.5,
    annotations=[
        dict(
            xref="paper",
            yref="paper",
            x=0.5,
            y=-.3,
            showarrow=False,
            text="Data source: Yahoo Finance / SEC"
        )
    ]
)

    fig.show()


        

if __name__ == '__main__':
    main()

[*********************100%%**********************]  1 of 1 completed
